In [124]:
#|default_exp AutonomousLLM

In [125]:
#|export
import os
import openai
import inspect
import types
import textwrap
import guardrails as gd
from rich import print
from dotenv import load_dotenv
from ast import literal_eval
import json
import nbdev
from retry import retry
import traceback
import subprocess
from functools import wraps
import types

In [126]:
#|export
load_dotenv()

True

# Code

In [135]:
#|export
def install_missing_module(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except ModuleNotFoundError as error:
            module_name = traceback.extract_tb(error.__traceback__)[-1].name
            print("Module not found:", module_name)

            # Install the module using pip
            try:
                subprocess.check_call(["pip", "install", module_name])
                print("Module installed successfully!")
                return func(*args, **kwargs)  # Retry the function after installation
            except subprocess.CalledProcessError as install_error:
                print("Failed to install the module:", install_error)
    
    return wrapper

In [136]:
#|export
class Utils:
    def __init__(self):
        pass

    def is_builtin(self, obj):
        """Check if an object is a built-in function or method."""
        if isinstance(obj, types.BuiltinFunctionType) or isinstance(
            obj, types.BuiltinMethodType
        ):
            return True
        return False

    def inspect_methods(self, obj):
        """Print the methods and code implementation of an object."""
        output_string = ""

        methods = inspect.getmembers(obj, inspect.ismethod)
        for name, method in methods:
            if not self.is_builtin(method):
                output_string += inspect.getsource(method) + "\n"

        dedented_code = textwrap.dedent(output_string)
        return dedented_code

    def convert_string_to_dict(self, string):
        cleaned_string = string.replace('\n', '')
        dictionary = json.loads(cleaned_string)
        return dictionary

In [137]:
#|export
class Call:
    def __init__(self) -> None:
        pass

    def _call_api(self, prompt):
        print("=== Calling API ====")
        print("PROMPT: ", prompt)
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt},
            ],
        )
        print(response.choices[0].message.content)
        return response
    
    def generate_code_given_task(self, task):
        prompt = f"""Based on a task, implement it in python code.
                    Ensure that the implementation is as detailed as possible, and ensure that it is fully functioning.
                    Only return python code and nothing else

                    The task is:
                    {task}"""
        
        return self._call_api(prompt)

In [138]:
#|export
class GuardedCall:

    def __init__(self) -> None:
        pass
    
    @retry(Exception, tries=3, delay=1)
    def _guard_call_api(self, guard, params={}, temperature = 0):
        print("=== Calling API ====")
        print("PARAMS: ", params)
        raw_llm_response, validated_response = guard(
            openai.ChatCompletion.create,
            prompt_params=params,
            model="gpt-3.5-turbo",
            max_tokens=2048,
            temperature=temperature,
        )

        print(raw_llm_response)

        return validated_response
    
    def format_code_to_JSON(self, code):
        guard = gd.Guard.from_rail("./rail/code_format.xml")
        return self._guard_call_api(guard, {"code": code})
     
    def generate_ideas(self):
        guard = gd.Guard.from_rail("./rail/idea_gen.xml")
        return self._guard_call_api(guard, temperature=0.7)
    
    def generate_subtasks_from_task(self, task):
        guard = gd.Guard.from_rail("./rail/subtask_gen.xml")
        return self._guard_call_api(guard, {'task': task} ,temperature=0.7)
        

In [139]:
#|export
class Chain:
    def __init__(self) -> None:
        pass

class CodeGenChain(Chain, GuardedCall, Call):
    def __init__(self) -> None:
        super().__init__()
    
    def run_pipeline(self, task: str):
        res = self.generate_code_given_task(task)
        res = self.format_code_to_JSON(res.choices[0].message.content)
        return res

In [148]:
#|export

class AutonomousLLM(Utils, GuardedCall, Call):
    def __init__(self):
        super().__init__()
        openai.api_key = os.getenv("OPENAI_API_KEY")
        self.ideas = []
        self.subtasks = []
        self.methods = []
        self.chain = CodeGenChain()

    @retry(Exception, tries=3, delay=1)
    @install_missing_module
    def execute_code(self, code):
        # Process the returned code and add it to AutonomousLLM
        exec(code, globals())

    @retry(Exception, tries=3, delay=1)
    @install_missing_module
    def execute_code_local(self, code):
        exec(code, globals(), locals())

    def add_ability(self, response):
        if new_func := response.get('new_method'):
            self.methods.append(new_func)  
            self.execute_code(textwrap.dedent(new_func))

        if attach_func := response.get('attach_method'):      
            self.execute_code_local(textwrap.dedent(attach_func))

    def run(self, num_calls=500):
        calls_to_make = num_calls
        while calls_to_make > 0:
            if len(self.subtasks) <= 0:
                calls_to_make -= 1
                # Make an API call to GPT-3.5 Turbo
                if len(self.ideas) < 1:
                    self.ideas += self.generate_ideas()['ideas']
                    continue
    
                idea = self.ideas.pop(0)
                print("IDEA:", idea)

                subtask_res = self.generate_subtasks_from_task(idea)

                if subtask_res.get('requires_subtasks') == True:
                    self.subtasks = subtask_res.get('subtasks')
                    continue
                
                res = self.chain.run_pipeline(idea)
                self.add_ability(res)
            else:
                subtask = self.subtasks.pop(0)
                res = self.chain.run_pipeline(subtask)
                self.add_ability(res)



    def use_new_ability(self):
        # Use the newly added ability in your code
        pass

In [153]:
#|export
auto_llm = AutonomousLLM()

In [154]:
#|export
auto_llm.run()

=== Calling API ====

PARAMS: 
{}

=== Calling API ====

PARAMS: 
{}

{
    "ideas": [
        "Ability to set reminders and receive notifications",
        "Integration with popular note-taking apps to save important information",
        "Personalized daily recommendations based on user preferences and habits"
    ]
}

...

1. Add a reminder

2. Delete a reminder

3. Print all reminders

4. Quit

ValueError: invalid literal for int() with base 10: ''

In [59]:
auto_llm.__dict__

{'ideas': ['Voice recognition for hands-free operation'],
 'subtasks': [],
 'methods': ['import datetime\n\nclass Task:\n    def __init__(self, name, priority, description, due_date):\n        self.name = name\n        self.priority = priority\n        self.description = description\n        self.due_date = datetime.datetime.strptime(due_date, \'%m/%d/%Y\')\n        self.completed = False\n\n    def complete_task(self):\n        self.completed = True\n\n    def __str__(self):\n        return f"Task Name: {self.name}\\nPriority: {self.priority}\\nDescription: {self.description}\\nDue Date: {self.due_date.strftime(\'%m/%d/%Y\')}\\nCompleted: {\'Yes\' if self.completed else \'No\'}\\n"\n\n\nclass TaskList:\n    def __init__(self):\n        self.tasks = []\n\n    def add_task(self, task):\n        self.tasks.append(task)\n\n    def remove_task(self, task_name):\n        for task in self.tasks:\n            if task.name.lower() == task_name.lower():\n                self.tasks.remove(task)\

AttributeError: 'AutonomousLLM' object has no attribute 'tasks'

# Testing

In [45]:
auto_llm.chain.run_pipeline("Add Google Maps integration to track user position")

=== Calling API ====

{
    "new_method": "def add_geolocation_map(self, api_key):\n    html = '''<!DOCTYPE html>\n<html>\n<head>\n    
<title>Google Maps Integration</title>\n    <style type=\"text/css\">\n        #map {\n            height: 100%;\n 
}\n        html, body {\n            height: 100%;\n            margin: 0;\n            padding: 0;\n        }\n   
</style>\n</head>\n<body>\n    <div id=\"map\"></div>\n    <script type=\"text/javascript\">\n        // Define the
initial center of the map\n        var initialCenter = {lat: 37.7749, lng: -122.4194};\n\n        // Load the 
Google Maps Javascript API\n        function loadMap() {\n            // Create a new map object\n            var 
map = new google.maps.Map(document.getElementById('map'), {\n                center: initialCenter,\n              
zoom: 10\n            });\n\n            // Create a new marker object to represent the user's position\n          
var marker = new google.maps.Marker({\n                position: initialCenter,\n                map: map,\n       
title: \"You are here\"\n            });\n\n            // Try HTML5 geolocation\n            if 
(navigator.geolocation) {\n                navigator.geolocation.getCurrentPosition(function(position) {\n         
// Set the marker's position to the user's current position\n                    var userPosition = {\n            
lat: position.coords.latitude,\n                        lng: position.coords.longitude\n                    };\n   
marker.setPosition(userPosition);\n\n                    // Center the map on the user's position\n                
map.setCenter(userPosition);\n                }, function() {\n                    // Handle geolocation errors\n  
alert(\"Geolocation failed.\");\n                });\n            } else {\n                // Browser doesn't 
support Geolocation\n                alert(\"Your browser does not support geolocation.\");\n            }\n       
}\n\n        // Load the map when the page loads\n        window.onload = loadMap;\n    </script>\n    <script 
src=\"https://maps.googleapis.com/maps/api/js?key={}\"></script>\n</body>\n</html>'''.format(api_key)\n    
self.geolocation_map = html",
    "attach_method": "setattr(self, 'add_geolocation_map', add_geolocation_map)"
}

{'new_method': 'def add_geolocation_map(self, api_key):\n    html = \'\'\'<!DOCTYPE html>\n<html>\n<head>\n    <title>Google Maps Integration</title>\n    <style type="text/css">\n        #map {\n            height: 100%;\n        }\n        html, body {\n            height: 100%;\n            margin: 0;\n            padding: 0;\n        }\n    </style>\n</head>\n<body>\n    <div id="map"></div>\n    <script type="text/javascript">\n        // Define the initial center of the map\n        var initialCenter = {lat: 37.7749, lng: -122.4194};\n\n        // Load the Google Maps Javascript API\n        function loadMap() {\n            // Create a new map object\n            var map = new google.maps.Map(document.getElementById(\'map\'), {\n                center: initialCenter,\n                zoom: 10\n            });\n\n            // Create a new marker object to represent the user\'s position\n            var marker = new google.maps.Marker({\n                position: initialCenter,\

# Analysis

In [121]:
for method in auto_llm.methods:
    print(method)

def identify_platforms():
    platforms = ["Alexa", "Google Home", "Apple HomeKit", "Microsoft Cortana", "Samsung SmartThings"]
    print("Platform(s) to integrate with:")
    for platform in platforms:
        print("- " + platform)

def add_method(self, function):
    setattr(self, function.__name__, function)

def add_method(self, func):
    setattr(self, func.__name__, func)

# Exec Testing

In [ ]:
globals_dict = {'x': 10}
locals_dict = {}

code_string = '''
y = x + 5
print(y)
'''

exec(code_string, globals_dict, locals_dict)

In [ ]:
locals_dict

# Guardrails

## Python Code

In [ ]:
rail_str = """
<rail version="0.1">

<output>
    <pythoncode
        name="python_code"
        format="bug-free-python"
        on-fail-bug-free-python="reask"
    />
</output>


<prompt>
Given the following high level leetcode problem description, write a short Python code snippet that solves the problem.

Problem Description:
{{leetcode_problem}}

@complete_json_suffix</prompt>

</rail>
"""

In [ ]:
import guardrails as gd

from rich import print

guard = gd.Guard.from_rail_string(rail_str)

In [ ]:
print(guard.base_prompt)

In [ ]:
import openai

leetcode_problem = """
Given a string s, find the longest palindromic substring in s. You may assume that the maximum length of s is 1000.
"""

raw_llm_response, validated_response = guard(
    openai.Completion.create,
    prompt_params={"leetcode_problem": leetcode_problem},
    engine="text-davinci-003",
    max_tokens=2048,
    temperature=0,
)

In [ ]:
raw_llm_response

In [ ]:
print(validated_response)

## ChatCompletions

In [ ]:
import guardrails as gd

from rich import print

content = gd.docs_utils.read_pdf("./chase_card_agreement.pdf")

print(f"Chase Credit Card Document:\n\n{content[:275]}\n...")

In [ ]:
rail_str = """
<rail version="0.1">

<output>
    <pythoncode
        name="new_method"
        format="bug-free-python"
        on-fail-bug-free-python="reask"
    />
    <pythoncode
        name="attach_method"
        format="bug-free-python"
        on-fail-bug-free-python="reask"
    />
</output>


<instructions>
You are a helpful assistant only capable of communicating with valid JSON, and no other text.

@json_suffix_prompt_examples
</instructions>


<prompt>
You are AutonomousLLM
Your goal is to continously self improve. The user will provide you with abilities to add to yourself.
When you receive such request, you must implement the requested ability in python code as a python function, you shall place this code in the 'new_method' key of the JSON object you return.
You shall use the setattr method to add a method to yourself. you shall place this code in the 'attach_method' key of the JSON object you return.
You shall place this code in the 'execute_method' key of the JSON object you return. 
Only return a JSON object and nothing else.

The user request is:
{{prompt}}

@xml_prefix_prompt

{output_schema}
</prompt>

</rail>
"""

In [ ]:
guard = gd.Guard.from_rail_string(rail_str)

In [ ]:
print(guard.base_prompt)

In [ ]:
print(guard.instructions.source)

In [ ]:
import openai

raw_llm_response, validated_response = guard(
    openai.ChatCompletion.create,
    prompt_params={"prompt": "Add the ability to detect sentiment of webpage"},
    model="gpt-3.5-turbo",
    max_tokens=2048,
    temperature=0,
)

In [ ]:
print(validated_response)

In [ ]:
raw_llm_response

In [ ]:
validated_response

In [ ]:
print(raw_llm_response)

In [120]:
auto_llm.methods

['def identify_platforms():\n    platforms = ["Alexa", "Google Home", "Apple HomeKit", "Microsoft Cortana", "Samsung SmartThings"]\n    print("Platform(s) to integrate with:")\n    for platform in platforms:\n        print("- " + platform)\n',
 'def add_method(self, function):\n    setattr(self, function.__name__, function)',
 'def add_method(self, func):\n    setattr(self, func.__name__, func)']

# Export

In [ ]:
nbdev.export.nb_export('./AutonomousLLM.ipynb', 'dist')

In [155]:
def save_code_to_file(code_list, file_path):
    with open(file_path, 'a') as file:
        file.write('\n')
        for code in code_list:
            file.write(code + '\n')

save_code_to_file(auto_llm.methods, "./outputs/methods.py")